In [7]:
# Imports
import pandas as pd
import numpy as np
import psycopg2, os

# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    

---
## 1. Restoring the DB

Follow below process to restore the data in Postgres:</br>

1. **Download dump**: Download file [here](https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc250/versao2019/postgis/)</br></br>
2. **Install `postgresql`**: If you don't have it installed you can run `brew install postgresql`</br></br>
3. **Restore the dump**_ Restore the data by running `pg_restore --host $AURORA_POSTGRES_HOST --port 5432 --username $AURORA_POSTGRES_USERNAME --dbname $AURORA_POSTGRES_DATABASE --no-owner --password --verbose "bc250_2019-10-29.tar"` (assuming you have the env variables configured)</br>

**After this process you will have the schema `bc250_2019` created at the `justos` DB.** Let's check tables created in the schema

In [10]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema = 'bc250_2019'")
pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])


,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,justos,bc250_2019,aer_pista_ponto_pouso_l,BASE TABLE,None,None,None,None,None,YES,NO,None
1,justos,bc250_2019,aer_pista_ponto_pouso_p,BASE TABLE,None,None,None,None,None,YES,NO,None
2,justos,bc250_2019,cbge_cemiterio_p,BASE TABLE,None,None,None,None,None,YES,NO,None
3,justos,bc250_2019,dut_trecho_duto_l,BASE TABLE,None,None,None,None,None,YES,NO,None
4,justos,bc250_2019,eco_ext_mineral_a,BASE TABLE,None,None,None,None,None,YES,NO,None
...,...,...,...,...,...,...,...,...,...,...,...,...
67,justos,bc250_2019,tra_ponte_p,BASE TABLE,None,None,None,None,None,YES,NO,None
68,justos,bc250_2019,tra_travessia_l,BASE TABLE,None,None,None,None,None,YES,NO,None
69,justos,bc250_2019,tra_travessia_p,BASE TABLE,None,None,None,None,None,YES,NO,None
70,justos,bc250_2019,tra_tunel_l,BASE TABLE,None,None,None,None,None,YES,NO,None


The [official documentation](https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc250/versao2019/informacoes_tecnicas/Documentacao_bc250_v2019.pdf) of this data dump can be used to understand the schema we have just restored.

---
